In [2]:
:dep polars={version="0.40.0", features = ["lazy","dtype-struct","csv"]}

In [3]:
use polars::prelude::*;
use std::fs::File;  // 为了用println!正确显示Polars

In [4]:
:dep plotters = {version="0.3.6", default_features = false, features = ["evcxr", "all_series", "all_elements"]}
use plotters::prelude::*;
use plotters::style::Color;

In [5]:
:dep colorous = {version="1.0.14"}
use colorous::*;

设置配色方案。

In [6]:
const TABLEAU20: [colorous::Color; 20] = [
    colorous::Color {r:31, g:119, b:180}, colorous::Color {r:174, g:199, b:232},
    colorous::Color {r:255, g:127, b:14}, colorous::Color {r:255, g:187, b:120},
    colorous::Color {r:44, g:160, b:44}, colorous::Color {r:152, g:223, b:138},
    colorous::Color {r:214, g:39, b:40}, colorous::Color {r:255, g:152, b:150},
    colorous::Color {r:148, g:103, b:189}, colorous::Color {r:197, g:176, b:213},
    colorous::Color {r:140, g:86, b:75}, colorous::Color {r:196, g:156, b:148},
    colorous::Color {r:227, g:119, b:194}, colorous::Color {r:247, g:182, b:210},
    colorous::Color {r:127, g:127, b:127}, colorous::Color {r:199, g:199, b:199},
    colorous::Color {r:188, g:189, b:34}, colorous::Color {r:219, g:219, b:141},
    colorous::Color {r:23, g:190, b:207}, colorous::Color {r:158, g:218, b:229}
];

导入数据：

In [7]:
let iris = LazyCsvReader::new("/data/wjw/hub/learn_rust/datasets/iris.csv").with_has_header(true).finish()?.collect()?;
println!("{}", iris.head(Some(3)));

shape: (3, 5)
┌──────────────┬─────────────┬──────────────┬─────────────┬─────────┐
│ Sepal.Length ┆ Sepal.Width ┆ Petal.Length ┆ Petal.Width ┆ Species │
│ ---          ┆ ---         ┆ ---          ┆ ---         ┆ ---     │
│ f64          ┆ f64         ┆ f64          ┆ f64         ┆ str     │
╞══════════════╪═════════════╪══════════════╪═════════════╪═════════╡
│ 5.1          ┆ 3.5         ┆ 1.4          ┆ 0.2         ┆ setosa  │
│ 4.9          ┆ 3.0         ┆ 1.4          ┆ 0.2         ┆ setosa  │
│ 4.7          ┆ 3.2         ┆ 1.3          ┆ 0.2         ┆ setosa  │
└──────────────┴─────────────┴──────────────┴─────────────┴─────────┘


设置 x、y 轴数据列及要用饼图表示的数据列：

In [8]:
let x = "Sepal.Length";
let y = "Sepal.Width";
let variable = ["Petal.Length", "Petal.Width"];
let pie_scale = 1.0;

设置饼图数据列对应的颜色：

In [9]:
let colormap: Vec<colorous::Color> = if variable.len() > 10 { TABLEAU20.to_vec() } else { CATEGORY10.to_vec() };
let colormap: Vec<RGBColor> = colormap.iter()
    .map(|c| RGBColor(c.as_tuple().0, c.as_tuple().1, c.as_tuple().2))
    .collect();
let mut colormap = colormap.into_iter().cycle();
let mut colors = vec![];
for _ in 0..variable.len() {
    colors.push(colormap.next().unwrap());
};

根据指定的列将所需的数据变成长型：

In [10]:
let dataset = iris.clone().lazy()
    .with_columns([
        col(x).cast(DataType::Float64),
        col(y).cast(DataType::Float64),
        cols(variable).cast(DataType::Float64),
    ])
    .with_row_index("ID", Some(0))
    .melt(MeltArgs {
        id_vars: vec!["ID".into(), x.into(), y.into()],
        value_vars: variable.to_vec().into_iter().map(|x| x.into()).collect::<Vec<_>>(),
        variable_name: Some("variable".into()), value_name: Some("value".into()), streamable: false,
    })
    .collect()?;

println!("{}", dataset.head(Some(3)));

shape: (3, 5)
┌─────┬──────────────┬─────────────┬──────────────┬───────┐
│ ID  ┆ Sepal.Length ┆ Sepal.Width ┆ variable     ┆ value │
│ --- ┆ ---          ┆ ---         ┆ ---          ┆ ---   │
│ u32 ┆ f64          ┆ f64         ┆ str          ┆ f64   │
╞═════╪══════════════╪═════════════╪══════════════╪═══════╡
│ 0   ┆ 5.1          ┆ 3.5         ┆ Petal.Length ┆ 1.4   │
│ 1   ┆ 4.9          ┆ 3.0         ┆ Petal.Length ┆ 1.4   │
│ 2   ┆ 4.7          ┆ 3.2         ┆ Petal.Length ┆ 1.3   │
└─────┴──────────────┴─────────────┴──────────────┴───────┘


设置饼图的默认半径和默认缩放因子，当数据中没有设置半径时就使用此默认半径。

In [11]:
let x_min = dataset.column(x)?.min::<f64>()?.unwrap();
let x_max = dataset.column(x)?.max::<f64>()?.unwrap();
let y_min = dataset.column(y)?.min::<f64>()?.unwrap();
let y_max = dataset.column(y)?.max::<f64>()?.unwrap();

let radius = (x_max - x_min)/50.0*pie_scale; // 半径默认为 x 轴范围的 2%
// x、y 轴向两边扩展一个半径长度，以免饼图出界
let mut x_start = x_min - radius;  
let mut x_end = x_max + radius;
let mut y_start = y_min - radius;
let mut y_end = y_max + radius;
// x、y 轴向两边再各扩展 5%，作为边界留白
let x_range = x_end - x_start;
let y_range = y_end - y_start;
x_start = x_start - x_range*0.05;
x_end = x_end + x_range*0.05;
y_start = y_start - y_range*0.05;
y_end = y_end + y_range*0.05;

设置作图区域大小和标度因子，将x、y的实际数值与图形像素大小对应起来。特别需要指出的是，图形使用的是像素坐标，左上角为坐标原点，而图表使用的是笛卡尔坐标，左下角是坐标原点。在进行坐标对应和转换的时候要注意到二者是不重合的，需进行特别处理。对本次应用场景而言，不需要作转换，二者的左上角都是坐标原点。

In [12]:
let area_width = 600u32;
let scale_factor = area_width as f64 / (x_end-x_start);
let area_height = (scale_factor * (y_end-y_start)) as u32;

利用循环提取每一个 ID 的信息（对应于原数据的每一行），将其放入一个向量。

In [13]:
// center 与 radius 需要调整
let id: Vec<u32> = dataset.column("ID")?.unique_stable()?.u32()?.into_no_null_iter().collect();
let mut pies = vec![];
for i in id.into_iter() {
    let data = dataset.clone().lazy()
        .filter(col("ID").eq(lit(i)))
        .collect()?;
    let x: Vec<f64> = data.column(x)?.f64()?.unique()?.into_no_null_iter().collect();
    let y: Vec<f64> = data.column(y)?.f64()?.unique()?.into_no_null_iter().collect();
    // 数学坐标转化为像素坐标，由于将浮点数转化为了整数，所以可能有细微的偏差，同时补偿一个半径，以免图像出界
    let center = ((
        ((x[0]-x_start)*scale_factor) as i32,
        ((y[0]-y_start)*scale_factor) as i32  // 像素坐标和位置坐标转换
    ));
    let sizes: Vec<f64> = data.column("value")?.f64()?.into_no_null_iter().collect();
    pies.push((center, radius*scale_factor, sizes));
}

()

In [15]:
evcxr_figure((area_width+100, area_height), |root| {
    // 将作图区域分割成两部分，右边宽度为100，放置图例
    let (left, right) = root.split_horizontally(area_width);
    for i in pies.iter() {
        let mut pie = Pie::new(&i.0, &i.1, &i.2, &colors, &variable);
        left.draw(&pie)?;
    }

    let mut legend_ctx = ChartBuilder::on(&right)
        //.margin_right(5)
        //.margin_top(50)
        //.set_label_area_size(LabelAreaPosition::Left, 0)
        //.set_label_area_size(LabelAreaPosition::Bottom, 60)
        .caption("legend", ("sans-serif", 25))
        .build_cartesian_2d(0.0..1.0, 0.0..1.0)
        .unwrap();
    // 绘制网格及坐标
    legend_ctx
        .configure_mesh()
        .set_all_tick_mark_size(0)
        .disable_x_axis()
        .disable_y_axis()
        .disable_x_mesh()
        .disable_y_mesh()
        .axis_style(BLACK)
        .label_style(("sans-serif", 20).into_font().color(&BLACK))
        .draw()
        .unwrap();
    // 产生数据点并绘制折线
    let mut colors = colors.iter();
    for i in variable.iter() {
        let color = colors.next().unwrap();
        // 绘制 bar
        legend_ctx
            .draw_series(vec![Circle::new((0.0, 0.0), 0, &WHITE)])?  // 点的直径设为 0 以不显示出来
            .label(format!("{}", i))  // format 使得 v 可以为 string 又可以为 &str
            // 下面 legend 的 move 是为了获取 color 的所有权，以免被下一个图例覆盖，必须使用，否则报错
            .legend(move |(x, y)| Rectangle::new([(x, y-6), (x+12, y+6)], color.mix(0.5).filled()));
    }
    legend_ctx
        .configure_series_labels()
        .position(SeriesLabelPosition::UpperLeft)
        .margin(5)
        .label_font(("Calibri", 25))  // 后面也可以加 into_font()或 into_text_style(&root)
        .draw()?;

    Ok(())
}).style("width:60%")

<polygon opacity="1" fill="#1F77B4" points="154,247 164,247 164,247 164,247 164,247 164,247 164,248 164,248 164,248 164,248 164,248 164,248 164,248 164,248 164,248 164,249 164,249 164,249 164,249 164,249 164,249 164,249 164,249 164,249 164,250 164,250 164,250 164,250 164,250 164,250 164,250 164,250 164,250 164,251 164,251 164,251 164,251 164,251 164,251 164,251 164,251 164,251 163,251 163,252 163,252 163,252 163,252 163,252 163,252 163,252 163,252 163,252 163,252 163,253 163,253 163,253 163,253 163,253 163,253 163,253 162,253 162,253 162,253 162,253 162,254 162,254 162,254 162,254 162,254 162,254 162,254 162,254 162,254 161,254 161,254 161,255 161,255 161,255 161,255 161,255 161,255 161,255 161,255 161,255 161,255 160,255 160,255 160,255 160,255 160,256 160,256 160,256 160,256 160,256 160,256 160,256 159,256 159,256 159,256 159,256 159,256 159,256 159,256 159,256 159,256 159,256 158,256 158,257 158,257 158,257 158,257 158,257 158,257 158,257 158,257 158,257 157,257 157,257 157,257 157,257 157,257 157,257 157,257 157,257 157,257 156,257 156,257 156,257 156,257 156,257 156,257 156,257 156,257 156,257 155,257 155,257 155,257 155,257 155,257 155,257 155,257 155,257 155,257 154,257 154,257 154,257 154,257 154,257 154,257 154,257 154,257 154,257 153,257 153,257 153,257 153,257 153,257 153,257 153,257 153,257 153,257 152,257 152,257 152,257 152,257 152,257 152,257 152,257 152,257 152,257 151,257 151,257 151,257 151,257 151,257 151,257 151,257 151,257 151,257 150,257 150,257 150,257 150,257 150,257 150,257 150,257 150,257 150,257 149,256 149,256 149,256 149,256 149,256 149,256 149,256 149,256 149,256 149,256 148,256 148,256 148,256 148,256 148,256 148,256 148,256 148,255 148,255 148,255 148,255 147,255 147,255 147,255 147,255 147,255 147,255 147,255 147,255 147,255 147,255 147,254 147,254 146,254 146,254 146,254 146,254 146,254 146,254 146,254 146,254 146,254 146,254 146,253 146,253 146,253 146,253 145,253 145,253 145,253 145,253 145,253 145,253 145,253 145,252 145,252 145,252 145,252 145,252 145,252 145,252 145,252 145,252 145,252 144,251 144,251 144,251 144,251 144,251 144,251 144,251 144,251 144,251 144,250 144,250 144,250 144,250 144,250 144,250 144,250 144,250 144,250 144,250 144,249 144,249 144,249 144,249 144,249 144,249 144,249 144,249 144,249 144,248 144,248 144,248 144,248 144,248 144,248 144,248 144,248 144,248 144,247 144,247 144,247 144,247 144,247 144,247 144,247 144,247 144,246 144,246 144,246 144,246 144,246 144,246 144,246 144,246 144,246 144,245 144,245 144,245 144,245 144,245 144,245 144,245 144,245 144,245 144,244 144,244 144,244 144,244 144,244 144,244 144,244 144,244 144,244 144,244 144,243 144,243 144,243 144,243 144,243 144,243 144,243 144,243 144,243 145,242 145,242 145,242 145,242 145,242 145,242 145,242 145,242 145,242 145,242 145,241 145,241 145,241 145,241 145,241 145,241 145,241 146,241 146,241 146,241 146,241 146,240 146,240 146,240 146,240 146,240 146,240 146,240 146,240 146,240 146,240 147,240 147,240 147,239 147,239 147,239 147,239 147,239 147,239 147,239 147,239 147,239 147,239 148,239 148,239 148,239 148,239 148,238 148,238 148,238 148,238 148,238 148,238 148,238 149,238 149,238 149,238 149,238 149,238 149,238 149,238 149,238 149,238 149,238 150,237 150,237 150,237 150,237 150,237 150,237 150,237 150,237 150,237 151,237 151,237 151,237 151,237 151,237 151,237 151,237 151,237 151,237 152,237 152,237 152,237 152,237 152,237 152,237 152,237 152,237 152,237 153,237 153,237 153,237 153,237 153,237 153,237 153,237 153,237 153,237 154,237 154,237 154,237 154,237 154,237 154,237 154,237 154,237 154,237 155,237 155,237 155,237 155,237 155,237 155,237 155,237 155,237 155,237 156,237 156,237 156,237 156,237 156,237 156,237 156,237 156,237 156,237 157,237 157,237 157,237 157,237 157,237 157,237 157,237 157,237 157,237 158,237 158,237 158,237 158,237 158,237 158,237 158,237 158,237 158,237 158,238 159,238 159,238 159,238 159,238 159,238 159,238 159,238 159,238 159,238 159,238 160,238 160,238 160,238 160,238 160,

参数说明：
* **pie_scale** amount to scale the pie size if there is no radius mapping exists.